In [1]:
import sys
import os
import numpy as np
import traceback

import sys 
sys.path.append("..")


In [2]:
from solvers.grid_search import GridSearchSolver
from solvers.search_space import SearchSpace, ContinuousParam, CategoricalParam


In [ ]:
try:
    class MockProblem:
        def __init__(self):
            self.name = "Mock"
            self.search_space = SearchSpace([
                ContinuousParam("x", 0.0, 1.0),
                CategoricalParam("c", ["A", "B"])
            ])
        
        def evaluate_objective(self, **kwargs):
            val = kwargs["x"]**2
            if kwargs["c"] == "B":
                val += 1.0
            return val

    print("Initializing Grid Solver...")
    problem = MockProblem()
    solver = GridSearchSolver(problem, n_points=3) # Grid: x=[0, 0.5, 1], c=[A, B]. Total 6 points.

    print("Running Grid Solver...")
    res = solver.run(evals=10) # Should evaluate all 6 points and stop or loop 
    # My implementation loops if evals > grid size
    
    print(f"Evaluated {res.n_evals} points.")
    print(f"Best: {res.x_best}, {res.y_best}")
    
    # Check if we found close to max? maximize=True by default. 
    # Max is x=1, c="B" -> 2.0
    
    assert res.y_best >= 2.0, f"Expected best >= 2.0, got {res.y_best}"
    print("Grid search test passed!")

except Exception:
    traceback.print_exc()
    sys.exit(1)


Initializing Grid Solver...
Running Grid Solver...
Evaluated 10 points.
Best: {'x': 1.0, 'c': 'B'}, 2.0
Grid search test passed!
